In [1]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer

# Recreate model architecture exactly as in training notebook
class FinbertBackbone(nn.Module):
    def __init__(self, modelName: str = "ProsusAI/finbert"):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(modelName)
        self.hiddenSize = self.encoder.config.hidden_size  # 768 for BERT-base

    def forward(self, input_ids, attention_mask):
        out = self.encoder(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        cls = out.last_hidden_state[:, 0]  # [CLS] token
        return cls  # [batch, hidden]

class BinaryHead(nn.Module):
    def __init__(self, inFeatures: int, pDrop: float = 0.1):
        super().__init__()
        self.dropout = nn.Dropout(pDrop)
        self.fc = nn.Linear(inFeatures, 1)  # single logit

    def forward(self, x):
        x = self.dropout(x)
        logits = self.fc(x).squeeze(-1)    # [batch]
        return logits

class FinbertBinaryClf(nn.Module):
    def __init__(self, modelName: str = "ProsusAI/finbert", pDrop: float = 0.1):
        super().__init__()
        self.backbone = FinbertBackbone(modelName)
        self.head = BinaryHead(self.backbone.hiddenSize, pDrop)

    def forward(self, input_ids, attention_mask):
        feats = self.backbone(input_ids, attention_mask)
        logits = self.head(feats)
        return logits


In [2]:
# Load tokenizer and model weights
model_name = "ProsusAI/finbert"
weights_path = "finbert_custom_head.pt"  # saved from training notebook

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = FinbertBinaryClf(modelName=model_name, pDrop=0.1).to(device)
state = torch.load(weights_path, map_location=device)
model.load_state_dict(state)
model.eval()

print(f"Device: {device}")
print("Model and tokenizer loaded.")


Device: cpu
Model and tokenizer loaded.


In [3]:
@torch.no_grad()
def predict_texts(texts, tokenizer, model, max_len=128, threshold=0.5, device=None):
    device = device or next(model.parameters()).device
    model.eval()
    if isinstance(texts, str):
        texts = [texts]
    enc = tokenizer(
        texts, truncation=True, padding="max_length", max_length=max_len, return_tensors="pt"
    )
    logits = model(enc["input_ids"].to(device), enc["attention_mask"].to(device))
    probs = torch.sigmoid(logits).cpu().numpy()
    labels = (probs >= threshold).astype(int)  # 1=positive, 0=negative
    return labels, probs


In [4]:
# Enter your text/tweet here
text = "Great earnings beat and strong guidance!"

labels, probs = predict_texts(text, tokenizer, model, max_len=128, threshold=0.5, device=device)
label = int(labels[0])
prob = float(probs[0])

sentiment_str = "positive - 1" if label == 1 else "negative - 0"
print(f"Input: {text}")
print(f"Prediction: {sentiment_str}")
print(f"Positive probability: {prob:.4f}")


Input: Great earnings beat and strong guidance!
Prediction: positive - 1
Positive probability: 0.9928


In [ ]:
# Optional: interactive prediction
while True:
    try:
        user_text = input("Enter a tweet/text to analyze: ")
        if user_text == "":
            break
        if user_text.strip():
            labels, probs = predict_texts(user_text, tokenizer, model, max_len=128, threshold=0.5, device=device)
            label = int(labels[0])
            prob = float(probs[0])
            sentiment_str = "positive - 1" if label == 1 else "negative - 0"
            print(f"input: {user_text}")
            print(f"Prediction: {sentiment_str}")
            print(f"Positive probability: {prob:.4f}")
        else:
            print("No text provided.")
    except EOFError:
        # In non-interactive environments, input() may fail
        pass


input: I think i’ll buy some $TSLA today before the close
Prediction: positive - 1
Positive probability: 0.9921
input: Meanwhile things continue to go well at Tesla. Wouldn't know it from the stock price however...
Prediction: positive - 1
Positive probability: 0.9909
input: TSLA now down $60 Pre-split in just a few hours today. Crazy stuff
Prediction: positive - 1
Positive probability: 0.6694
input: Thank you @Tesla for delivering a car to the wrong address (my house), blocking my driveway for hours and not allowing me to pickup my kids from school on such a hot day. On top of that, all your driver had to say was ""call Tesla and tell them, I'm just a driver""
Prediction: negative - 0
Positive probability: 0.3421
input: This thing is never coming to market
Prediction: negative - 0
Positive probability: 0.0676
input: Cybertruck will be able to serve as a boat.   This product is going to be nuts, for real.
Prediction: positive - 1
Positive probability: 0.8089
No text provided.
